# **TextTorch - 02: Representação de Texto**

**Objetivo:** Converter os textos pré-processados em vetores numéricos. Este notebook implementa a lógica para **TF-IDF** (padrão) e inclui uma seção comentada para **embeddings**, permitindo a troca fácil entre as duas abordagens.

### **Como Executar (Google Colab)**

1. **Pré-requisito:** Execute o notebook `01_preprocessing.ipynb` para gerar o arquivo `artifacts/processed_dataset.pkl`.
2. **Ambiente:** Se estiver em um novo ambiente, clone o repositório e instale as dependências.
   ```bash
   !git clone https://github.com/takaokensei/TextTorch.git
   %cd TextTorch
   !pip install -r requirements.txt
   ```
3. **Execução:** Execute todas as células deste notebook.

In [1]:
# Imports e Configurações Iniciais
import sys
import os
import pickle
import torch
import numpy as np

# Adiciona o diretório 'src' ao path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from representation import TFIDFRepresentation, sparse_to_dense_tensors
from model import load_config

# Carrega configurações
CONFIG_PATH = '../models/config.yaml'
config = load_config(CONFIG_PATH)

# Carrega dataset processado
DATASET_PATH = '../artifacts/processed_dataset.pkl'
with open(DATASET_PATH, 'rb') as f:
    dataset = pickle.load(f)

print("Ambiente e dataset carregados.")

2025-11-13 17:26:51,951 - INFO - Configuração carregada de: ../models/config.yaml


Ambiente e dataset carregados.


## 1. Representação TF-IDF (Padrão)

A abordagem padrão utiliza **TF-IDF (Term Frequency-Inverse Document Frequency)** para converter textos em vetores. Esta técnica é eficiente e funciona bem como um baseline robusto.

- **`fit_transform`**: Treina o `TfidfVectorizer` no conjunto de treino.
- **`transform`**: Aplica o vectorizer já treinado nos conjuntos de validação e teste.

In [2]:
# Inicializa o vectorizer com parâmetros do config
tfidf_repr = TFIDFRepresentation(
    min_df=config['min_df'],
    max_df=config['max_df'],
    ngram_range=tuple(config['ngram_range']),
    max_features=config['max_features']
)

# Treina no conjunto de treino e transforma
X_train_tfidf = tfidf_repr.fit_transform(dataset['X_train'])

# Apenas transforma os conjuntos de validação e teste
X_val_tfidf = tfidf_repr.transform(dataset['X_val'])
X_test_tfidf = tfidf_repr.transform(dataset['X_test'])

print(f"Dimensão do vetor TF-IDF (features): {tfidf_repr.input_dim}")
print(f"Shape da matriz de treino: {X_train_tfidf.shape}")

2025-11-13 17:26:55,119 - INFO - Treinando TF-IDF em 252 documentos...
2025-11-13 17:26:55,146 - INFO - TF-IDF treinado: 676 features
2025-11-13 17:26:55,147 - INFO - Esparsidade: 95.34%


Dimensão do vetor TF-IDF (features): 676
Shape da matriz de treino: (252, 676)


### Análise das Features TF-IDF

Podemos inspecionar as features (palavras e n-gramas) que o `TfidfVectorizer` aprendeu.

In [3]:
top_features = tfidf_repr.get_top_features(n=20)
print("--- Top 20 Features por IDF (mais raras e informativas) ---")
for feature, score in top_features:
    print(f"- {feature}: {score:.2f}")

--- Top 20 Features por IDF (mais raras e informativas) ---
- vetor: 4.74
- a inovacao: 4.74
- uma revolucao: 4.74
- ambiental e: 4.74
- elementos: 4.74
- ecoturismo: 4.74
- envolvem: 4.74
- entre diferentes: 4.74
- energia: 4.74
- empregos: 4.74
- atletas de: 4.74
- as possibilidades: 4.74
- as estrategias: 4.74
- territorial: 4.74
- tecnologicas: 4.74
- em infraestrutura: 4.74
- e valorizacao: 4.74
- e tecnologias: 4.74
- e sociais: 4.74
- e servicos: 4.74


### Salvando o Vectorizer

Salvamos o objeto `TfidfVectorizer` treinado para uso posterior na avaliação e no deploy.

In [4]:
VECTORIZER_PATH = '../artifacts/vectorizer.joblib'
tfidf_repr.save(VECTORIZER_PATH)

2025-11-13 17:27:15,350 - INFO - Vectorizer salvo em: ../artifacts/vectorizer.joblib


## 2. Conversão para Tensores PyTorch

O modelo PyTorch espera tensores como entrada. Convertemos as matrizes esparsas do TF-IDF em tensores densos.

In [5]:
# Converte matrizes esparsas para tensores densos
X_train_tensor = sparse_to_dense_tensors(X_train_tfidf)
X_val_tensor = sparse_to_dense_tensors(X_val_tfidf)
X_test_tensor = sparse_to_dense_tensors(X_test_tfidf)

# Converte labels para tensores
y_train_tensor = torch.tensor(dataset['y_train'], dtype=torch.long)
y_val_tensor = torch.tensor(dataset['y_val'], dtype=torch.long)
y_test_tensor = torch.tensor(dataset['y_test'], dtype=torch.long)

print(f"Shape do tensor de treino: {X_train_tensor.shape}")
print(f"Shape do tensor de labels de treino: {y_train_tensor.shape}")

Shape do tensor de treino: torch.Size([252, 676])
Shape do tensor de labels de treino: torch.Size([252])


## 3. Salvando os Tensores

Salvamos os tensores processados para uso nos próximos notebooks.

In [6]:
TENSORS_PATH = '../artifacts/tensors_tfidf.pt'

torch.save({
    'X_train': X_train_tensor,
    'X_val': X_val_tensor,
    'X_test': X_test_tensor,
    'y_train': y_train_tensor,
    'y_val': y_val_tensor,
    'y_test': y_test_tensor,
    'input_dim': tfidf_repr.input_dim,
    'n_classes': dataset['n_classes']
}, TENSORS_PATH)

print(f"Tensores TF-IDF salvos em: {TENSORS_PATH}")

Tensores TF-IDF salvos em: ../artifacts/tensors_tfidf.pt


---

## **OPCIONAL: Representação com Embeddings**

**Para ativar esta seção:**
1.  **Altere a configuração:** Em `models/config.yaml`, mude `representation: tfidf` para `representation: embedding`.
2.  **Descomente o código:** Descomente a classe `EmbeddingRepresentation` em `src/representation.py` e o código nas células abaixo.
3.  **Reexecute este notebook.**

Esta abordagem aprende uma representação densa para cada palavra, o que pode capturar melhor as relações semânticas.

In [ ]:
# --- CÓDIGO PARA EMBEDDINGS (DESCOMENTE PARA USAR) ---

# from representation import EmbeddingRepresentation

# if config['representation'] == 'embedding':
#     print("\n--- Iniciando Pipeline de Embeddings ---")
    
#     # 1. Construir vocabulário
#     embedding_repr = EmbeddingRepresentation(
#         vocab_size=config.get('vocab_size', 20000),
#         embedding_dim=config.get('embedding_dim', 300)
#     )
#     embedding_repr.build_vocabulary(dataset['X_train'])
    
#     # 2. Converter textos para sequências de índices
#     X_train_seq = embedding_repr.texts_to_sequences(dataset['X_train'])
#     X_val_seq = embedding_repr.texts_to_sequences(dataset['X_val'])
#     X_test_seq = embedding_repr.texts_to_sequences(dataset['X_test'])
    
#     # Labels já são tensores
#     y_train_tensor_emb = torch.tensor(dataset['y_train'], dtype=torch.long)
#     y_val_tensor_emb = torch.tensor(dataset['y_val'], dtype=torch.long)
#     y_test_tensor_emb = torch.tensor(dataset['y_test'], dtype=torch.long)
    
#     print(f"Shape do tensor de treino (embeddings): {X_train_seq.shape}")
    
#     # 3. Salvar vocabulário e tensores
#     VOCAB_PATH = '../artifacts/vocab.joblib'
#     embedding_repr.save(VOCAB_PATH)
    
#     TENSORS_EMB_PATH = '../artifacts/tensors_embedding.pt'
#     torch.save({
#         'X_train': X_train_seq,
#         'X_val': X_val_seq,
#         'X_test': X_test_seq,
#         'y_train': y_train_tensor_emb,
#         'y_val': y_val_tensor_emb,
#         'y_test': y_test_tensor_emb,
#         'vocab_size': embedding_repr.vocab_size,
#         'n_classes': dataset['n_classes']
#     }, TENSORS_EMB_PATH)
    
#     print(f"Vocabulário salvo em: {VOCAB_PATH}")
#     print(f"Tensores de embedding salvos em: {TENSORS_EMB_PATH}")